# 문장을 작은 단위로 쪼개기

## 2-1 토큰화란?

**토큰화**란 문장을 **토큰 시퀀스**로 나누는 과정
 문자 단어 서브워드 등의 방법이 있다.
 ### 단어 단위 토큰화
- 단어(어절)단위
- 공백으로 분리하여 토크나이저 안써도됨
- 어휘 집합의 크기가 커짐 
### 문자 단위 토큰화
- 해당 언어의 모든 문자를 어휘집화
  - 미등록 토큰 문제가 없음(신조어에 유리)
- 각 문자 토큰은 의미있는 단위가 되기 어려움
- 토큰의 시퀀스 길이가 길어짐
### 서브워드 단위 토큰화
- 단어와 문자 돤위의 중간
- 예: 바이트 페어 인코딩


## 2-2 바이트 페어 인코딩이란?


**바이트 페어 인코딩** 정보를 압축하는 알고리즘에서 최초 사용
GPT=BPE사용
Bert=워드피스 사용
### BPE란
데이터에서 가장 많이 사용한 **문자열을 병합**해서 데이터를 압축하는 기법
- aaabdaaabac
- ZabdZabac
- ZYdZTac
- XdXac

사전 크기 증가를 억제하면서도 정보를 효율적으로 압축할 수 있는 알고리즘
### BPE 어휘집 구축하기
**고빈도 바이그램 쌍을 병합하는 방식으로 구축함**
1. 프리토크나이즈(말뭉치의 모든 문장을 공백으로 나누기/다른 기준도 OK)
  - h,u,g: 10
2.토큰을 2개씩 묶어서 나열( 쌍이 같은 것 끼리 합치기 포함)
3. 가장 많이 등장한 바이그램 쌍을 어휘 집합에 추가
4. 1부터 다시 반복(어취집합의 크기가 사용자 지정에 도달할 때 까지)
  - 어휘 집합(voca.json) 구축 결과
    - b,g,h,n,p,s,u,ug,un,hug
  -병합 우선순위(merge.text)병합 결과 기록 저장한것
    - u g
    - u n
    - h ug

### BPE 토큰화
어휘 집합(voca.json)과 병합 우선순위(merge.text)를 가지고 함
예
  pug, bug, mug
  p, ug, b, ug, \<unk\>, ug
일반적으로 개별문자(알파벳)등은 처음 어휘 집합에 들어가므로 **미등록 토큰이 거의 없다**.=> 신조어에 유의미한 분절 수행가능
### 워드피스
말뭉치에서 자주 등장한 문자열 토큰화(BPE와 유사)
But**말뭉치의 우도를 가장 높이는 쌍을 병합**
a,b,ab라는 문자열이 n의 글자수에서 나오는 것 가정
(#ab/n)/(#a/n)*((#b/n)
- a,b가 독립일때, 위식에 값이 커야 동시에 등장 함


## 2-3 어휘 집합 구축하기


1,2단계

In [4]:
!pip install ratsnlp

In [5]:
#드라이브 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


3단계 말뭉치 내려받기 및 전처리

In [6]:
#말뭉치 내려받기 및 전처리
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)
##말뭉치는 박은정 님이 공개하신 Naver Sentiment Movie Corpus(NSMC)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 134MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 71.4MB/s]


In [7]:
#NSMC 전처리
import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')

write_lines("/content/train.txt", nsmc.train.get_all_texts())
write_lines("/content/test.txt", nsmc.test.get_all_texts())

#NSMC에 포함된 영화 리뷰(순수 텍스트)들을 지정된 경로에 저장

In [8]:
#!head train.txt
#!head test.txt

4단계 GPT토크나이저 구축
GPT 계열 모델이 사용하는 토크나이저는 Byte-level Byte Pair Encoding(BBPE)
**유니바코드 수준으로 어휘 집합을 구축하고 토큰화를 수행**
한글의 경우 3개의 유니코드바이트로 한 글자가 표현됨

In [9]:
#디렉토리 만들기
import os
os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok=True)

In [10]:
# nsmc 데이터를 가지고 BBPE 어휘집합을 구축
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],#학습 말뭉치를 리스트 형태로 넣기
    vocab_size=10000,#어휘 집합의 크기
    special_tokens=["[PAD]"] #특수 토큰 추가
)
bytebpe_tokenizer.save_model("/gdrive/My Drive/nlpbook/bbpe")

## vocab.json은 바이트 레벨 BPE의 어휘 집합
## merges.txt는 바이그램 쌍의 병합 우선순위


['/gdrive/My Drive/nlpbook/bbpe/vocab.json',
 '/gdrive/My Drive/nlpbook/bbpe/merges.txt']

In [ ]:
# vocab.json
# !cat /gdrive/My\ Drive/nlpbook/bbpe/vocab.json
# merges.txt
# !head /gdrive/My\ Drive/nlpbook/bbpe/merges.txt

5 단계 BERT토크나이저 구축

BERT는 워드피스(wordpiece) 토크나이저를 사용

In [11]:
# 디렉터리 만들기
import os
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok=True)

In [12]:
#워드피스 어휘 집합 구축
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
)
wordpiece_tokenizer.save_model("/gdrive/My Drive/nlpbook/wordpiece")

['/gdrive/My Drive/nlpbook/wordpiece/vocab.txt']

In [13]:
#구축 결과
#!head /gdrive/My\ Drive/nlpbook/wordpiece/vocab.txt

## 2-4 토큰화하기


문장을 토큰화 + 토큰을 모델에 입력

In [ ]:
!pip install ratsnlp

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


2단계 GPT 입력값 만들기

In [28]:
#GPT 모델 입력값을 만들려면 Byte-level Byte Pair Encoding 
#어휘집합 구축 결과(vocab.json, merges.txt)가 
#자신의 구글 드라이브 경로(/gdrive/My Drive/nlpbook/wordpiece)에 있어야 함

#토크나이저 선언
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file /gdrive/My Drive/nlpbook/bbpe/config.json not found


In [29]:
#토크나이저로 토큰화 하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]
##토큰화결과
# tokenized_sentences

In [30]:
# 모델 입력 만들기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",#문장의 최대 길이에 맞춰 패딩
    max_length=12,#문장의 토큰 기준 최대 길이
                  #문장1,3에는 뒤에 [PAD]토큰에 해당하는 인덱스0이 붙는다.
                  #[PAD]는 일종의 더미 변수임
    truncation=True,#문장 잘림 허용 옵션
                    #문장 2의 경우 15 토큰이여야 하지만 12로 잘렸고 이를 허용하는 것임
)
## 배치의 크기가 3이라 가정

**input_ids**와 **attention_mask**가 만들어짐
전자는 토큰화 결과를 가지고 각 토큰을 인텍스로 바꾼것(인덱싱 한것)
어휘집(vocab.jason)을 확인해보면 어휘 순서대로 나열 확인 이 순서가 인덱스임

후자는 일반토큰이 자리한곳(1)과 패딩 토큰이 자리한 곳을 구분해주는 장치


In [31]:

print(batch_inputs.keys())
batch_inputs['input_ids']

dict_keys(['input_ids', 'attention_mask'])


[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [32]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

3단계 BERT 입력값 만들기

BERT 모델 입력값을 만들려면 자신의 구글 드라이브 경로(/gdrive/My Drive/nlpbook/wordpiece)에 워드피스 어휘집합 구축 결과(vocab.txt)가 있어야

In [19]:
#BERT 토크나이저 선언
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/My Drive/nlpbook/wordpiece", 
    do_lower_case=False,
)

file /gdrive/My Drive/nlpbook/wordpiece/config.json not found


In [20]:
#BERT 토크나이저로 토큰화 하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]
#tokenized_sentences

In [21]:
#모델 입력 만들기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

In [22]:
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [23]:
batch_inputs['input_ids']
#문장 처음에 2([CLS]) 끝에는 3([SEP])이 붙음


[[2, 620, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1214, 16, 3],
 [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [24]:
batch_inputs['attention_mask']
#일반 토큰이 자리한곳(1)과 패딩 토큰이 자리한 곳(0)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [25]:
batch_inputs['token_type_ids']
#세그먼트에 해당하는 것으로 모두0
#버트는 보통 2개의 문서를 입력 받음 이때 token_type_ids로 구분함
#0이면 1번 문서 1이면 2번 문서임

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]